In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import accuracy_score
import keras
from keras.applications import VGG19,Xception,VGG16
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from keras import layers

In [ ]:
# import  train test data 
# loading training set

train_datagen = ImageDataGenerator(
    rescale = 1/255,
    shear_range = 0.2,
    zoom_range = 0.2,
    vertical_flip =  True, 
    rotation_range = 40,
    brightness_range = (0.5, 1.5),
    horizontal_flip = True
)

train_data = train_datagen.flow_from_directory(
    '/content/skin-cancer-malignant-vs-benign/train',
    target_size = (64, 64),
    class_mode='sparse',
    shuffle = True,
    seed = 1
)

## loading validation dataset

test_datagen = ImageDataGenerator(rescale = 1/255)

test_data = test_datagen.flow_from_directory(
    '/content/skin-cancer-malignant-vs-benign/test',
    target_size = (64, 64),
    class_mode='sparse',
    shuffle = True,
    seed = 1
)

In [ ]:
class_names = ["Benign", "Malignant"]
for i in class_names :
    print(class_names.index(i),"  ",i)

In [ ]:
#visulaize test_data

fig, ax = plt.subplots()
ax.bar(["benign"], [300], color = "r", label = 'benign')
ax.bar(["malignat"], [360], color = "b", label = "malignat")
ax.legend()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
image_path = "/content/skin-cancer-malignant-vs-benign/test/benign/1261.jpg"
new_img = image.load_img(image_path, target_size = (244, 244))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis = 0)
print("benign")
plt.imshow(new_img)

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
image_path = "/content/skin-cancer-malignant-vs-benign/test/malignant/1080.jpg"
new_img = image.load_img(image_path, target_size = (244, 244))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis = 0)
print("malignant")
plt.imshow(new_img)

In [ ]:
## Defining CNN

model = tf.keras.models.Sequential([
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.3),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Conv2D(256, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.15),
    layers.Dense(2, activation= 'softmax')
])

In [ ]:
#compile the model
import tensorflow as tf
model.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])

#early stopping function
early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

#fit the model
histroy = model.fit(
    train_data,
    validation_data = test_data, 
    callbacks = [early],
    epochs = 5
)

In [ ]:
#evulate model
model.evaluate(test_data)